# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import datasets
import torch
import transformers

from collections import Counter
from torch.nn import CrossEntropyLoss

In [ ]:
#| hide
from datasets import load_dataset
from nbdev.showdoc import *
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
#| export
def loss_func(logits, labels):
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss_fct = CrossEntropyLoss(reduction="none")
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    return loss

In [ ]:
#| export
def get_counts(model, tokenizer, batch, semantic_column):
    input_ids = torch.tensor(batch["input_ids"])
    attention_mask = torch.tensor(batch["attention_mask"])
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids, return_dict=True)
    loss = loss_func(outputs.logits, input_ids)
    # Add the losses to the counter for each 
    # token in the input
    loss_cnt = Counter()
    token_cnt = Counter()
    for i, token in enumerate(input_ids[1:]): # Skip the first token since labels are shifted
        token = tokenizer.decode(token)
        loss_cnt[token] += loss[i].item()
        token_cnt[token] += 1
    
        if semantic_column != None:
            semantic = batch[semantic_column][i]
            loss_cnt[semantic] += loss[i].item()
            token_cnt[semantic] += 1
    return loss_cnt, token_cnt

In [ ]:
#| export
def perplexed(
    model: transformers.PreTrainedModel, # The model to calculate the perplexity of.
    dataset: datasets.Dataset, # The dataset to calculate the perplexity on.
    tokenizer: transformers.PreTrainedTokenizer = None, # The tokenizer to use to tokenize the dataset. If not provided, the tokenizer associated with the model will be used.
    column: str = "text", # The column of the dataset to calculate the perplexity on.
    semantic_column: str = None, # The column of the dataset to calculate the semantic perplexity on such as NER tags.
    n_gram: int = 1, # The n-gram to calculate the perplexity on.
    batch_size: int = 1, # The batch size to use when calculating the perplexity.
    device: str = "cuda", # The device to use when calculating the perplexity.
    return_tokens: bool = False, # Whether to return the tokens counts along with the perplexity.
): # The perplexity of the model on the dataset or a tuple of the perplexity and the token counts.
    """
    Calculate the perplexity of a model on a dataset.
    """
    if tokenizer is None:
        tokenizer = model.config.tokenizer_class.from_pretrained(model.config.pretrained_model_name_or_path)

    # Tokenize the dataset
    tokenized_dataset = dataset.map(
        lambda x: tokenizer(x[column], return_tensors="pt", truncation=True),
        batched=True,
        batch_size=batch_size,
        remove_columns=dataset.column_names,
    )

    # TODO: Add support for semantic perplexity

    # Calculate the perplexity of the model on the dataset
    total_loss_cnt = Counter()
    total_token_cnt = Counter()
    for batch in tokenized_dataset:
        # calculate the loss for each token
        loss_cnt, token_cnt = get_counts(model, tokenizer, batch, semantic_column)
        # add the loss and token counts to the total
        total_loss_cnt += loss_cnt
        total_token_cnt += token_cnt
    
    # Calculate the perplexity
    perplexity_cnt = Counter()
    for token, loss in total_loss_cnt.items():
        perplexity_cnt[token] = torch.exp(torch.tensor(loss / total_token_cnt[token])).item()
    
    if return_tokens:
        return perplexity_cnt, total_token_cnt
    
    return perplexity_cnt

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test").select(range(1_000))
# filter out empty strings
dataset = dataset.filter(lambda x: len(x["text"]) > 0)

perplexity_cnt, token_cnt = perplexed(model, dataset, tokenizer=tokenizer, column="text", batch_size=1, device="cpu", return_tokens=True)

In [ ]:
perplexity_cnt.most_common(10)

In [ ]:
tokens, _ = zip(*perplexity_cnt.most_common(10))
# show the token counts for the most perplexed tokens
for token in tokens:
    print(token, token_cnt[token])

In [ ]:
# top least common tokens
perplexity_cnt.most_common()[-10:]

In [ ]:
tokens, _ = zip(*perplexity_cnt.most_common()[-10:])
# show the token counts for the least perplexed tokens
for token in tokens:
    print(token, token_cnt[token])

In [ ]:
# most common token
token_cnt.most_common(10)

In [ ]:
tokens, _ = zip(*token_cnt.most_common(10))
# show the perplexity for the most common tokens
for token in tokens:
    print(token, perplexity_cnt[token])

In [ ]:
# the least common token
token_cnt.most_common()[-10:]

In [ ]:
tokens, _ = zip(*token_cnt.most_common()[-10:])
# show the perplexity for the least common tokens
for token in tokens:
    print(token, perplexity_cnt[token])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()